In [2]:
import os
import json
import numpy as np
import pandas as pd
import re

repo = '<path to repo>'

# Model checking plan data

In [3]:
def extract_plan_data(run):
    data = []
    file = open(repo+'/{}/autoctrl.txt'.format(run), 'r')
    
    for line in file:
        if line.split(':')[0] == 'ACCEPT':
            data.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'PATH':
            data.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'PLAN':
            data.append(line.split(':')[-1].strip())
        elif line.strip()[-2:] == 'ms':
            d = line.split('!')[1].strip()[:-2]
            data.append(float(d))
            
    file.close()
    
    if len(data) == 0:
        data = [None] * 8
    
    if len(data) == 4:
        data += [None] * 4

    return data
            
def add_plan_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
            run_data = json.load(infile)

        log_data = extract_plan_data(run)
        run_data['accept1'] = log_data[0]
        run_data['path1'] = log_data[1]
        run_data['plan1'] = log_data[2]
        if (log_data[2] != None):
            run_data['steps1'] = int(len(log_data[2].split('->')[:-1]))
        else:
            run_data['steps1'] = log_data[2]
        run_data['latency1'] = log_data[3]
        run_data['accept2'] = log_data[4]
        run_data['path2'] = log_data[5]
        run_data['plan2'] = log_data[6]
        if (log_data[6] != None):
            run_data['steps2'] = int(len(log_data[6].split('->')[:-1]))
        else:
            run_data['steps2'] = log_data[6]
        run_data['latency2'] = log_data[7]

        with open(repo+'/{}/{}.json'.format(run, run), 'w') as outfile: 
            json.dump(run_data, outfile)
            
        count += 1
      
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))
    
add_plan_to_run_data(n_runs=90)

# Model checking resource usage

In [3]:
def _extract_stack_data(data):
    result = []
    
    for item in data:
        if item[:3] != 'SET':
            result.append(item.strip())
        else:
            break
            
    result = result[2:-2]
    # get stack size data here
    result = [int(x[-1]) for x in result]
            
    return result
    
def _extract_set_data(data):
    result = []
    
    for item in reversed(data[:-5]):
        if item[:3] != 'SET':
            try:
                result.append(int(item.strip()))
            except ValueError:
                pass
        else:
            break
        
    return result
    
def extract_performance_data(run):
    perf = []
    
    for file in os.listdir(repo+'\{}'.format(run)):
        if file[:3] == 'dfs':
            with open(repo+'\{}\{}'.format(run, file)) as infile:
                perf.append(infile.readlines())
         
    set_data = []
    stack_data = []
    
    for data in perf:
        stack_data += _extract_stack_data(data)
        set_data += _extract_set_data(data)
    
    return stack_data, set_data

def add_performance_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
            run_data = json.load(infile)

        stack_data, set_data = extract_performance_data(run)
        run_data['state_size'] = 4
        run_data['n_states'] = 15
        run_data['adj_list_compile'] = 184
        run_data['stack_compile'] = 12
        run_data['set_compile'] = 24
        
        if len(stack_data) > 0:
            # max stack size and number of steps
            run_data['max_stack_capacity'] = max(stack_data)
        else:
            run_data['max_stack_capacity'] = None
            
        if len(set_data) > 0:
            run_data['max_set_size'] = max(set_data)
        else:
            run_data['max_set_size'] = None
            
        with open(repo+'/{}/{}.json'.format(run, run), 'w') as outfile: 
            json.dump(run_data, outfile)
            
        count += 1
      
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))

add_performance_to_run_data(n_runs=90)

# Process resource usage

In [5]:
def _extract_process_memory(run):
    swap = []
    physical = []
    process = []
    
    file = open(repo+'/{}/usage.txt'.format(run), 'r')
    
    for line in file:
        if line.split(':')[0] == 'MiB Mem ':
            physical.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'MiB Swap':
            swap.append(line.split(':')[-1].strip())
        elif line.strip()[-8:] == 'autoctrl':
            process.append(line)

    file.close()

    # physical data
    physical_ = []
    physical = [x.split(',') for x in physical]
    for row in physical:
        physical_.append([float(x.strip()[:4]) for x in row])

    # swap data
    swap_ = []
    swap = [x.split(',') for x in swap]
    for row in swap:   
        swap_.append([x.strip() for x in row])

    swap__ = []
    for row in swap_:
        swap__.append([float(x.split()[0]) for x in row])

    # process data
    process_ = []
    for row in process:   
        process_.append(row.strip()[9:-8].strip().split(' '))

    process__ = []
    for row in process_:
        process__.append([x for x in row if x != ''])

    for i in range(len(process__)):    
        for j in range(len(process__[i])):
            if ':' in process__[i][j]:
                process__[i][j] = process__[i][j][-5:]
                
            try:
                process__[i][j] = float(process__[i][j])
            except ValueError:
                pass
                
    phy_cols = ['mem_total', 'mem_free', 'mem_used', 'cache']
    phy = pd.DataFrame(data=physical_, columns=phy_cols)

    swp_cols = ['swap_total', 'swap_free', 'swap_used']
    swp = pd.DataFrame(data=swap__, columns=swp_cols)

    proc_cols = ['pr', 'ni', 'virt', 'res', 'shr', 'status', 
                 'cpu_perc', 'mem_perc', 'cpu_time']
    proc = pd.DataFrame(data=process__, columns=proc_cols)

    df = pd.merge(proc, phy, how='inner', left_index=True, right_index=True)
    df = pd.merge(df, swp, how='inner', left_index=True, right_index=True)
    df['process_uptime'] = 0.001 * df.index
    df = df[df.mem_perc > 0]
    
    return df

def add_process_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
            run_data = json.load(infile)

        df = _extract_process_memory(run)
        df.to_csv(repo+'/{}/{}usage.csv'.format(run, run))

        run_data['virt_min'] = df.virt.min()
        run_data['virt_max'] = df.virt.max()
        run_data['res_min'] = df.res.max()
        run_data['res_max'] = df.res.max()
        run_data['shr_min'] = df.shr.min()
        run_data['shr_max'] = df.shr.max()
        run_data['mem_perc_min'] = df.mem_perc.min()
        run_data['mem_total_min'] = df.mem_total.min()
        run_data['mem_perc_max'] = df.mem_perc.max()
        run_data['mem_total_max'] = df.mem_total.max()
        run_data['mem_free_min'] = df.mem_total.min()
        run_data['mem_free_max'] = df.mem_total.max()
        run_data['mem_used_min'] = df.mem_used.min()
        run_data['mem_used_max'] = df.mem_used.max()
        run_data['cache_min'] = df.cache.min()
        run_data['cache_max'] = df.cache.max()
        run_data['swap_total_min'] = df.swap_total.min()
        run_data['swap_total_max'] = df.swap_total.max()
        run_data['swap_free_min'] = df.swap_free.min()
        run_data['swap_free_max'] = df.swap_free.max()
        run_data['swap_used_min'] = df.swap_used.min()
        run_data['swap_used_max'] = df.swap_used.max()
        run_data['process_uptime'] = df.process_uptime.max()

        with open(repo+'/{}/{}.json'.format(run, run), 'w') as outfile: 
            json.dump(run_data, outfile)

        count += 1
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))

add_process_to_run_data(n_runs=90)